# A model-driven approach to building AI agents in just a few lines of code

Strands Agents is a simple yet powerful SDK that takes a model-driven approach to building and running AI agents. From simple conversational assistants to complex autonomous workflows, from local development to production deployment, Strands Agents scales with your needs.<br/><br/>
This notebook demonstrates how to leverage the **Strands Agents SDK and Tools** to generate and execute automated Python code across a diverse range of tasks.

### Suppress Warnings for Cleaner Output  
This cell imports the `warnings` module and suppresses warnings to ensure the notebook output remains clean and easy to read.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### Install Strands Agents Libraries  
Installs `strands-agents` and related tools for creating Python agents and tools that can run and evaluate Python code.


In [ ]:
!pip install strands-agents strands-agents-tools strands-agents-builder nest_asyncio

In [ ]:
# Optionally install additional libraries that are needed for the use cases. 
# If you do not have them, LLM will identify missing libraries and will try to install in the Python REPL environment.
!pip install yfinance matplotlib
!pip install beautifulsoup4 pandas requests

## Use Case 1: Web Scraping with a few lines of code
Uses `requests` and `BeautifulSoup` to scrape article titles and links from Hacker News, saving results to a CSV file.


In [ ]:
from strands_tools import python_repl, file_write
from strands import Agent, tool
import os

os.environ["DEV"] = "true"

agent = Agent(tools=[python_repl, file_write])

prompt = """
Get the titles and the links of the articles listed in https://news.ycombinator.com/news. 
When you execute a python script, make sure to run it in non-interactive mode.
Write the results as a CSV file names news_{date}.csv.
"""

response = agent(prompt)

## Use Case 2: Stock Price Analysis  
Downloads historical stock data, calculates moving averages, key financial metrics, and buy/sell signals with visualizations and logging for Apple Inc. using `yfinance` and `matplotlib`.


In [ ]:
from strands_tools import current_time, python_repl
from strands import Agent, tool
import os

os.environ["DEV"] = "true"

system_prompt = """
You are a financial analyst. Provide following charts:
- Plot 20-days moving average of closing prices for the past one year.
- Plot daily return rate comparison against S&P500 for the same period.

Also, compute following two metrics of the stock:
  - Volatility of the return rates

Use `yfinance` module to retrive the historical data.
"""

agent = Agent(tools=[python_repl, current_time],
              system_prompt=system_prompt,
              model="anthropic.claude-3-5-sonnet-20241022-v2:0")

response = agent("Amazon")

### Use Case 3: Weather conditions inspection
In this scenario, we will use an agent which investigate if there was a storm in particular county duing specified time. The validated result is to be stored in AWS DynamoDB table for downstream applications.

In [ ]:
from strands_tools import use_aws, http_request, current_time, think
from strands import Agent, tool
import os

os.environ["DEV"] = "true"

system_prompt = """
You are a specialized insurance claim inspector. We have insurance claims getting from home owners, who claim their houses were damaged by severe weather condition.
Investigate if there was any severe storm which can damage a house in their area then summarize your findings and store it into a DynanoDB table.

[Instructions]
- Use GET request to https://www.ncdc.noaa.gov/stormevents/csv search the weather database. Refer an example URL provided below.
- Write the results into a DynamoDB table "WeatherInspection" in us-west-2. Create the table if not exists. Use ClaimID as a partitian key, inspection date as a sort key.
"""

example_url = """Follwing URL is an example to find weather events in ALAMEDA county in CALIFORNIA from 2024-01-20 to 2024-12-31
    https://www.ncdc.noaa.gov/stormevents/csv?eventType=ALL&beginDate_mm=01&beginDate_dd=20&beginDate_yyyy=2024&endDate_mm=12&endDate_dd=31&endDate_yyyy=2024&county=ALAMEDA%3A1&statefips=6%2CCALIFORNIA"""

agent = Agent(tools=[use_aws, http_request, current_time, think],
              system_prompt=f"{system_prompt} {example_url}",
              model="anthropic.claude-3-5-sonnet-20240620-v1:0"
            )


response = agent("Find if there was a severe weather in Wheeler County, GA in Aug-2024.")

### Use Case 4: Text-to-DSL using MCP - Reusing the code from the previous Lab
Let us repeat Text-to-DSL use case we worked in the previous lab. This time, we will use Strands SDK instead of LangGraph.<br/>
<br/>
**You should have completed the previous Text-to-DSL Lab to run this lab.**

In [ ]:
from strands import Agent
from strands.tools.mcp import MCPClient
from mcp import stdio_client, StdioServerParameters
import boto3
from IPython.display import display, Markdown

# Define a system prompt
SYSTEM_PROMPT = """
Write a DSL query to retrive data from guardduty-index for the following question. 
Check the schema of the index to write accurate and concise queries. 

Format your answer as a Markdown report consists of four sections:
1. A DSL query that was used.
2. Summarized answer for the question.
3. Tabular format of the findings.
4. Recommended remediations based on your knowledge
"""
credentials = boto3.Session().get_credentials()
server_params = StdioServerParameters(
        command="python",  # Executable
        args=["mcp_dsl_server.py"],  # Optional command line arguments
        env={
            "AWS_ACCESS_KEY_ID": credentials.access_key,
            "AWS_SECRET_ACCESS_KEY": credentials.secret_key,
            "AWS_SESSION_TOKEN": credentials.token
        },  # Optional environment variables
    )

# Load an MCP server that can query OpenSearch Index with DSL
opensearch_tool = MCPClient(lambda: stdio_client(server_params))

with opensearch_tool:
    agent = Agent(
        system_prompt=SYSTEM_PROMPT,
        tools=opensearch_tool.list_tools_sync(),
        model = "anthropic.claude-3-5-sonnet-20241022-v2:0"
    )

    display(Markdown(agent("Any crypto currency mining activity happening in our clusters?").message["content"][0]["text"]))

### Use Case 5: Bring your own Python script - LLM usage calculator
This example will show how to import your own Python functions as the tools that your agent can access. <br/>
Please refer ```llm_cost_utils.py``` file which defines ```get_bedrock_token_count``` function with the ```@tool``` decorator.

In [ ]:
from llm_cost_utils import get_bedrock_token_count
from strands import Agent
from strands_tools import current_time
from IPython.display import Markdown, display

SYSTEM_PROMPT="""
Find token count for an LLM model given using the tool provided for given time. Then calculate the cost of the token usage.
"""

agent = Agent(tools=[get_bedrock_token_count, current_time])

display(Markdown(agent("Cost of us.anthropic.claude-3-7-sonnet-20250219-v1:0 usage during 24 hours?").message["content"][0]["text"]))